## Data Analysis

In [2]:
import re

# OEB

In [3]:
import re

# File paths
input_file = 'OEB.txt'
output_tsv_file = 'OEB_combined.tsv'

# Book names extracted from the table of contents
book_names = [
    "Ruth", "Esther", "Psalms", "Hosea", "Joel", "Amos", "Obadiah", "Jonah", 
    "Micah", "Nahum", "Habakkuk", "Zephaniah", "Haggai", "Zechariah", "Malachi",
    "Matthew", "Mark", "Luke", "John", "Acts", "Romans", "1 Corinthians", 
    "2 Corinthians", "Galatians", "Ephesians", "Philippians", "Colossians", 
    "1 Thessalonians", "2 Thessalonians", "1 Timothy", "2 Timothy", "Titus", 
    "Philemon", "Hebrews", "James", "1 Peter", "2 Peter", "1 John", "2 John", 
    "3 John", "Jude", "Revelation"
]

# Regex pattern to detect verses
verse_pattern = re.compile(r"\[(\d+:\d+)\]\s+(.+?)(?=\[\d+:\d+\]|$)")

# Data collection
combined_data = []
current_book = None

# Use the order of books from the table of contents to infer context
book_index = 0

with open(input_file, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()

        # Detect verses
        for verse_match in verse_pattern.finditer(line):
            verse_id, verse_text = verse_match.groups()
            chapter_num, verse_num = verse_id.split(":")

            # Assign current book based on the detected order in the text
            if current_book is None or len(combined_data) > 0 and int(chapter_num) == 1 and int(verse_num) == 1:
                current_book = book_names[book_index]
                book_index += 1

            # Collect data
            combined_data.append((current_book, chapter_num, verse_num, verse_text.strip()))

# Write the data to the TSV file
with open(output_tsv_file, 'w', encoding='utf-8') as out_file:
    out_file.write("Book\tChapter\tVerse\tText\n")
    for entry in combined_data:
        out_file.write("\t".join(entry) + "\n")

print(f"Extraction complete! {len(combined_data)} verses written to {output_tsv_file}")


Extraction complete! 11722 verses written to OEB_combined.tsv


# WEB

In [4]:
# Unified processing for the World English Bible (WEB)

input_web_file = 'WEB.txt'
output_web_tsv_file = 'WEB_combined.tsv'

# Regex patterns for books, chapters, and verses
book_pattern = re.compile(r"^Book \d+\s+(.+)$")  # Matches lines like "Book 01 Genesis"
verse_pattern = re.compile(r"^(\d{3}):(\d{3})\s+(.+)$")  # Matches lines like "001:001 Text"

# Data collection
web_data = []
current_book = None

with open(input_web_file, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()

        # Detect book titles
        book_match = book_pattern.match(line)
        if book_match:
            current_book = book_match.group(1).strip()
            continue

        # Detect verses
        verse_match = verse_pattern.match(line)
        if verse_match and current_book:
            chapter_num, verse_num, verse_text = verse_match.groups()
            web_data.append((current_book, chapter_num, verse_num, verse_text.strip()))

# Write data to a single TSV file
with open(output_web_tsv_file, 'w', encoding='utf-8') as out_file:
    out_file.write("Book\tChapter\tVerse\tText\n")
    for entry in web_data:
        out_file.write("\t".join(entry) + "\n")

len(web_data), output_web_tsv_file


(31102, 'WEB_combined.tsv')

## KJV

In [6]:
import re

# Input and output files
input_kjv_file = 'KJV.txt'
output_ot_file = 'KJV_OT.tsv'
output_nt_file = 'KJV_NT.tsv'

# Markers for Old and New Testaments
ot_marker = "The Old Testament of the King James Version of the Bible"
nt_marker = "The New Testament of the King James Bible"

# Verse pattern: Matches "Chapter:Verse Text"
verse_pattern = re.compile(r"(\d+):(\d+)\s+(.+)")

# Data containers
ot_data = []
nt_data = []
current_testament = None
current_text = []
current_chapter, current_verse = None, None

with open(input_kjv_file, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()

        # Detect Testament markers
        if ot_marker in line:
            current_testament = 'OT'
            continue
        elif nt_marker in line:
            current_testament = 'NT'
            continue

        # Detect verses
        verse_match = verse_pattern.match(line)
        if verse_match:
            # Save the previous verse
            if current_chapter and current_verse:
                full_text = " ".join(current_text).strip()
                if current_testament == 'OT':
                    ot_data.append((current_chapter, current_verse, full_text))
                elif current_testament == 'NT':
                    nt_data.append((current_chapter, current_verse, full_text))

            # Start a new verse
            current_chapter, current_verse, text = verse_match.groups()
            current_text = [text]
        else:
            # Accumulate lines for the current verse
            current_text.append(line)

    # Save the last verse
    if current_chapter and current_verse:
        full_text = " ".join(current_text).strip()
        if current_testament == 'OT':
            ot_data.append((current_chapter, current_verse, full_text))
        elif current_testament == 'NT':
            nt_data.append((current_chapter, current_verse, full_text))

# Write data to TSV files
with open(output_ot_file, 'w', encoding='utf-8') as ot_file:
    ot_file.write("Chapter\tVerse\tText\n")
    for entry in ot_data:
        ot_file.write("\t".join(entry) + "\n")

with open(output_nt_file, 'w', encoding='utf-8') as nt_file:
    nt_file.write("Chapter\tVerse\tText\n")
    for entry in nt_data:
        nt_file.write("\t".join(entry) + "\n")

print(f"Old Testament verses: {len(ot_data)}")
print(f"New Testament verses: {len(nt_data)}")

Old Testament verses: 18963
New Testament verses: 6032
